<a id="top"></a>
#### QCT. Practices with Adiabatic Quantum Computing

# Problem Formulation

#### Lecturers:
- Rafael Martín-Cuevas Redondo, rafamartinc@pm.me

***

In this notebook, we will model some simple problems using D-Wave's Ocean SDK, as a first contact with their environment and tools, as well as with QUBO and Ising models.

### Table of Contents

1. [QUBO Models](#qubo)
2. [Ising Models](#ising)
3. [Ising to QUBO](#ising_to_qubo)
4. [QUBO to Ising](#qubo_to_ising)
5. [Benchmarking different solvers from D-Wave Systems](#dwave_solvers)

***

Please run the following block to import the libraries that will be required during the execution of this notebook. If your system lacks any of the libraries mentioned, use ```pip install -r requirements.txt``` in a command line, within this notebook's folder, to ensure that all required libraries are installed.

In [1]:
import time
import numpy
from pyqubo import Spin, Binary
import qubogen

import dimod
import neal

[Back to the top](#top)

***
<a id="qubo"></a>

## 1. QUBO Models.
### First example: 4 warehouses, 3 couriers

In [2]:
qubo = {
    ('x1', 'x1'): -2,
    ('x2', 'x2'): -1,
    ('x3', 'x3'): -4,
    ('x1', 'x2'): 2
}
num_reads = 5000

sampler = neal.SimulatedAnnealingSampler()
response = sampler.sample_qubo(qubo, num_reads=num_reads)
response = response.aggregate()

for values, _, num_occurrences in response.data():
    variables = [key for key in values if values[key] != 0]
    print('Solution {}: {}/{} occurrences'.format(variables, num_occurrences, num_reads))

Solution ['x1', 'x3']: 4937/5000 occurrences
Solution ['x2', 'x3']: 30/5000 occurrences
Solution ['x1', 'x2', 'x3']: 32/5000 occurrences
Solution ['x3']: 1/5000 occurrences


[Back to the top](#top)

***
<a id="ising"></a>

## 2. Ising Models.
### Second example: 6 warehouses, 4 couriers

In [3]:
h = {
    's1': 1,
    's2': 1,
    's3': -2,
    's4': 0
}
J = {
    ('s1', 's2'): 2,
    ('s1', 's4'): 2,
    ('s2', 's4'): 1
}
num_reads = 5000

sampler = neal.SimulatedAnnealingSampler()
response = sampler.sample_ising(h, J, num_reads=num_reads)
response = response.aggregate()

for values, _, num_occurrences in response.data():
    variables = [key for key in values if values[key] != -1]
    print('Solution {}: {}/{} occurrences'.format(variables, num_occurrences, num_reads))

Solution ['s3', 's4']: 1664/5000 occurrences
Solution ['s2', 's3', 's4']: 1663/5000 occurrences
Solution ['s1', 's3']: 1665/5000 occurrences
Solution ['s1', 's3', 's4']: 4/5000 occurrences
Solution ['s2', 's3']: 4/5000 occurrences


[Back to the top](#top)

***
<a id="ising_to_qubo"></a>

## 3. Ising to QUBO

In this section, we will try to convert the Ising model from the second example into a QUBO model, using the open-source library [PyQUBO](https://github.com/recruit-communications/pyqubo), to check the solution found in the slides with the traditional method.

In [4]:
s1 = Spin('s1')
s2 = Spin('s2')
s3 = Spin('s3')
s4 = Spin('s4')

H = 2*s1*s2 + 2*s1*s4 + s2*s4 + s1 + s2 - 2*s3 + 7
ising = H.compile()
qubo, offset = ising.to_qubo()

print(qubo, offset)

{('s1', 's1'): -6.0, ('s2', 's2'): -4.0, ('s2', 's1'): 8.0, ('s2', 's4'): 4.0, ('s3', 's3'): -4.0, ('s1', 's4'): 8.0, ('s4', 's4'): -6.0} 12.0


[Back to the top](#top)

***
<a id="qubo_to_ising"></a>

## 4. QUBO to Ising

In this section, we will try to convert the QUBO model from the first example into an Ising model, using the open-source library [PyQUBO](https://github.com/recruit-communications/pyqubo), to check the solution found in the slides with the traditional method.

In [7]:
x1 = Binary('x1')
x2 = Binary('x2')
x3 = Binary('x3')

H = 2*x1*x2 - 2*x1 - x2 - 2*x3 + 4
qubo = H.compile()
h, J, offset = qubo.to_ising()

print(h, J, offset)

{'x1': -0.5, 'x3': -1.0} {('x1', 'x2'): 0.5} 2.0


[Back to the top](#top)

***
<a id="dwave_solvers"></a>

## 5. Benchmarking different solvers from D-Wave Systems

In this section, we will try to convert the QUBO model from the first example into an Ising model, using the open-source library [PyQUBO](https://github.com/recruit-communications/pyqubo), to check the solution found in the slides with the traditional method.

In [8]:
cases = [
    {
        'variables': num_variables + 1,
        'bqm': dimod.BQM(
            qubogen.qubo_number_partition(
                numpy.random.randint(1, 10, num_variables + 1)
            ),
            'BINARY'
        )
    } for num_variables in range(24)
]

print('Runtimes with different solvers:')
print('   1. neal.SimulatedAnnealingSampler')
print('   2. dimod.SimulatedAnnealingSampler')
print('   3. dimod.ExactSolver')

for case in cases:

    start_time = time.time()
    sampler.sample(case['bqm'])
    time_nealsas = time.time() - start_time

    start_time = time.time()
    dimod.SimulatedAnnealingSampler().sample(case['bqm'])
    time_dimodsas = time.time() - start_time

    start_time = time.time()
    dimod.ExactSolver().sample(case['bqm'])
    time_dimodes = time.time() - start_time
    
    print('{} variable(s): {} {} {}'.format(case['variables'], time_nealsas, time_dimodsas, time_dimodes))

Runtimes with different solvers:
   1. neal.SimulatedAnnealingSampler
   2. dimod.SimulatedAnnealingSampler
   3. dimod.ExactSolver
1 variable(s): 0.009283065795898438 0.023881912231445312 0.0009999275207519531
2 variable(s): 0.0 0.042784929275512695 0.0
3 variable(s): 0.002005338668823242 0.07349109649658203 0.0


c:\Users\Alvaro\anaconda3\envs\dwave\lib\site-packages\dwave\samplers\sa\sampler.py:524: UserWarning: All bqm biases are zero (all energies are zero), this is likely a value error. Temperature range is set arbitrarily to [0.1,1]. Metropolis-Hastings update is non-ergodic.
  warnings.warn(warn_msg)


4 variable(s): 0.000997781753540039 0.11153697967529297 0.0
5 variable(s): 0.0020024776458740234 0.15599465370178223 0.0
6 variable(s): 0.0010037422180175781 0.21086740493774414 0.0
7 variable(s): 0.0019876956939697266 0.27260351181030273 0.0
8 variable(s): 0.0010058879852294922 0.34403204917907715 0.0
9 variable(s): 0.0025246143341064453 0.42103147506713867 0.0009989738464355469
10 variable(s): 0.0010097026824951172 0.5134532451629639 0.0009982585906982422
11 variable(s): 0.002521991729736328 0.5980560779571533 0.003477334976196289
12 variable(s): 0.0009984970092773438 0.7268562316894531 0.006544828414916992
13 variable(s): 0.0009891986846923828 0.8291397094726562 0.012995481491088867
14 variable(s): 0.0010013580322265625 0.9405455589294434 0.02661442756652832
15 variable(s): 0.0010061264038085938 1.0709583759307861 0.05045962333679199
16 variable(s): 0.0029942989349365234 1.2125577926635742 0.12145781517028809
17 variable(s): 0.002018451690673828 1.3570685386657715 0.2650127410888672